## Spark SQL es muy fácil de usar, y punto. Quizá ya sepas que también es bastante difícil de dominar.

Para ser competente en Spark, hay que tener tres habilidades fundamentales:

-La capacidad de manipular y entender los datos
-El conocimiento de cómo adaptar la herramienta a las necesidades del programador
-El arte de encontrar un equilibrio entre los factores que afectan a las ejecuciones de los trabajos de Spark

He elaborado los siguientes seis ejercicios que se asemejan a algunas situaciones típicas a las que los desarrolladores de Spark se enfrentan a diario cuando construyen sus pipelines: estos ayudarán a evaluar las habilidades anteriores.

### El conjunto de datos
Describamos brevemente el conjunto de datos que vamos a utilizar: consta de tres tablas procedentes de la base de datos de una tienda, con productos, ventas y vendedores. Los datos están disponibles en archivos Parquet


<center><img src="https://miro.medium.com/max/700/1*wA4xJu3LMcm_vR5pFJkLpA.png" width=500 height=500 />
    



### Tabla de ventas
Cada fila de esta tabla es un pedido y cada pedido puede contener sólo un producto. Cada fila almacena los siguientes campos:

- order_id: El ID del pedido
- product_id: El único producto vendido en el pedido. Todos los pedidos tienen exactamente un producto)
- seller_id: El ID del empleado vendedor que vendió el producto
- num_pieces_sold: El número de unidades vendidas para el producto específico en el pedido
- bill_raw_text: Una cadena que representa el texto en bruto de la factura asociada al pedido
- date: La fecha del pedido.

### Tabla de productos
Cada fila representa un producto distinto. Los campos son

- product_id: El ID del producto
- product_name: El nombre del producto
- price: El precio del producto

### Tabla de vendedores
Esta tabla contiene la lista de todos los vendedores:

- seller_id: El ID del vendedor
- seller_name: El nombre del vendedor
- daily_target: El número de artículos (independientemente del tipo de producto) que el vendedor necesita para alcanzar su cuota. Por ejemplo, si el objetivo diario es 100.000, el empleado necesita vender 100.000 productos, puede alcanzar la cuota vendiendo 100.000 unidades del producto_0, pero también vendiendo 30.000 unidades del producto_1 y 70.000 unidades del producto_2


### Ejercicios
La mejor manera de aprovechar los ejercicios que se presentan a continuación es obtener los datos e implementar un código de trabajo que resuelva los problemas propuestos.

Consejo: He construido el conjunto de datos para poder trabajar en una sola máquina: cuando escribas el código, imagina lo que pasaría con un conjunto de datos 100 veces mayor.

Aunque sepas cómo resolverlas, ¡mi consejo es que no te saltes las preguntas de calentamiento! (si sabes que Spark te llevará unos segundos).


In [1]:
# Pyspark
!pip install pyspark pyspark-stubs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.1/209.1 MB 5.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 5.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of pyspark-stubs to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.3 MB/s eta 0:00:0

### 1.0 ¿Cuál es el ingreso medio de los pedidos?

Recuerde que los ingresos = precio * cantidad. (revenue = price * quantity.)

In [ ]:
#Solución ejercicio 1.0
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.files.overwrite", "true") \
    .appName("Exercise1") \
    .getOrCreate()


products = spark.read.csv(
   'file:///clase8/products.csv', header=True, mode="DROPMALFORMED"
)

sellers = spark.read.csv(
   'file:///clase8/sellers.csv', header=True, mode="DROPMALFORMED"
)

sales = spark.read.csv(
   'file:///clase8/sales.csv', header=True, mode="DROPMALFORMED"
)

##### A partir de aquí escriba su respuesta





spark.stop()

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|            1245.9236386027228|
+------------------------------+

None


### 2.0  Para cada vendedor, ¿cuál es el porcentaje medio de contribución de un pedido a la cuota diaria del vendedor?

#### Ejemplo
Si el Seller_0 con `quota=250`  tiene 3 pedidos:

- Order  1: 10 productos vendidos
- Order  2: 8 productos vendidos
- Order  3: 7 productos vendidos

El porcentaje medio de contribución de los pedidos a la cuota del vendedor sería
- Order  1: 10/250 = 0,04
- Order  2: 8/250 = 0,032
- Order  3: 7/250 = 0,028

Porcentaje medio de contribución = (0,04+0,032+0,028)/3 = 0,03333

In [ ]:
#Solución ejercicio 2.0
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.files.overwrite", "true") \
    .appName("Exercise1") \
    .getOrCreate()


products = spark.read.csv(
   'file:///clase8/products.csv', header=True, mode="DROPMALFORMED"
)

sellers = spark.read.csv(
   'file:///clase8/sellers.csv', header=True, mode="DROPMALFORMED"
)

sales = spark.read.csv(
   'file:///clase8/sales.csv', header=True, mode="DROPMALFORMED"
)

##### A partir de aquí escriba su respuesta



spark.stop()

+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        3|7.060842894390445E-4|
|        7|8.510553537464244E-5|
|        8|0.002071646546208...|
|        0|2.019736225290412...|
|        1|3.670188787905763...|
|        2|2.456721945951509E-4|
|        4|3.845384604576898...|
|        5|8.038980497173663E-5|
|        6|2.534518215186249...|
|        9|1.449276275189615E-4|
+---------+--------------------+

None


### 3.0  ¿Quiénes son las segundas personas (vendedores) que más venden y las que menos venden para cada producto?

Recomendación: use la función `Window` [link de ayuda](https://sparkbyexamples.com/spark/spark-sql-window-functions/)

In [ ]:
#Solución ejercicio 3.0
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.files.overwrite", "true") \
    .appName("Exercise1") \
    .getOrCreate()


products = spark.read.csv(
   'file:///clase8/products.csv', header=True, mode="DROPMALFORMED"
)

sellers = spark.read.csv(
   'file:///clase8/sellers.csv', header=True, mode="DROPMALFORMED"
)

sales = spark.read.csv(
   'file:///clase8/sales.csv', header=True, mode="DROPMALFORMED"
)

##### A partir de aquí escriba su respuesta




spark.stop()

+----------+---------+-----------------+
|product_id|seller_id|             type|
+----------+---------+-----------------+
|  19581081|        1|     Least Seller|
|   3093518|        1|     Least Seller|
|   3147422|        1|     Least Seller|
|   3384033|        1|     Least Seller|
|  57452470|        2|     Least Seller|
|  25078796|        4|     Least Seller|
|  42503932|        4|     Least Seller|
|  49196172|        4|     Least Seller|
|  70135114|        4|     Least Seller|
|  24992315|        5|     Least Seller|
|  46764734|        5|     Least Seller|
|  36289301|        6|     Least Seller|
|  55049289|        6|     Least Seller|
|  57356373|        6|     Least Seller|
|  22113759|        7|     Least Seller|
|  44179664|        7|     Least Seller|
|   5342885|        7|     Least Seller|
|  54810407|        7|     Least Seller|
|  74608973|        7|     Least Seller|
|  19657520|        8|     Least Seller|
|  63583705|        8|     Least Seller|
|  28365122|    